<h1>1. Установка среды Марио</h1>

In [ ]:
#   Правообладатель gym-super-mario-bros
#   @misc{gym-super-mario-bros,
#   author = {Christian Kauten},
#   howpublished = {GitHub},
#   title = {{S}uper {M}ario {B}ros for {O}pen{AI} {G}ym},
#   URL = {https://github.com/Kautenja/gym-super-mario-bros},
#   year = {2018},

In [ ]:
import gym_super_mario_bros #загружаем игру
from nes_py.wrappers import JoypadSpace #загружаем Joypad.Wrapper
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT #загрузка управления

In [ ]:
#установка игры
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
# #цикл для запуска игры
# done = True 
# #шаг в каждый шаг обучения
# for step in range(50000):
#     if done:
#         #начало игры
#         state = env.reset()
#     state, reward, done, info = env.step(env.action_space.sample()) #рандомайзер действий
#     env.render() #показ на экран

# env.close()

<h1>2. Подготовка среды</h1>

In [ ]:
from gym.wrappers import GrayScaleObservation #FrameStack нужен для отправления в стек предыдущих шагов цикла
#GrayScaleObservation нужен для уменьшения количества данных(цвета) для ускорения работы
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
#работа с векторами (векторизация)
from matplotlib import pyplot as plt

In [ ]:
#установка основной среды
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Упрощение комманд
env = JoypadSpace(env, SIMPLE_MOVEMENT)
#Эффект серого
env = GrayScaleObservation(env, keep_dim=True)
#Dummy Environment
env = DummyVecEnv([lambda: env])
#Отправка кадров в стэк
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
state = env.reset()

In [ ]:
state, reward, done, info = env.step([5]) #рандомайзер действий

In [ ]:
plt.figure(figsize=(20,16))
for i in range(state.shape[3]):
    plt.subplot(1,4,i+1)
    plt.imshow(state[0][:,:,i])
plt.show()

<h1>3. Испытание модели обучения с подкреплением(RL)</h1>

In [ ]:
import os #для оперирования с путями (path management)
from stable_baselines3 import PPO #PPO обладет некоторыми дополнительными алгоритмами машинного обучения
from stable_baselines3.common.callbacks import BaseCallback #для сохранения модели каждые n раз (бэкап)

In [ ]:
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
#Сохранение нашей модели каждые n раз (в нашем случае 10 000)
callback = TrainAndLoggingCallback(check_freq=50000, save_path=CHECKPOINT_DIR)

In [ ]:
#Запуск Модели
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512) 

In [ ]:
# Начало Обучения Модели
model.learn(total_timesteps=1000000, callback=callback)

<h1>4. Тестирование Модели</h1>

In [ ]:
# Заргузка модели
model = PPO.load('./train/best_model_50000')

In [ ]:
# Начало Игры
state = env.reset()
# Цикл по игре
while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()